In [1]:
from PIL import Image, ImageDraw
import torch
from torchvision.transforms import functional as F
from yolov5.models.experimental import attempt_load
from yolov5.utils.general import non_max_suppression

WARNING  'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING  'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
# Example: Load multiple YOLOv5 models
model1 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model2 = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)
model3 = torch.hub.load('ultralytics/yolov5', 'yolov5l', pretrained=True)

model = model.to('cpu')
model.eval()

Using cache found in C:\Users\su00n/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-6 Python-3.11.3 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Using cache found in C:\Users\su00n/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-6 Python-3.11.3 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Using cache found in C:\Users\su00n/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-6 Python-3.11.3 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 
Using cache found in C:\Users\su00n/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-6 Python-3.11.3 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [3]:
img_path = './input_ref.jpg'
img = Image.open(img_path).convert('RGB')
img = img.resize((640, 640), Image.LANCZOS)
img_tensor = F.to_tensor(img).unsqueeze(0)

In [4]:
results_model = []
with torch.no_grad():
    # Get raw predictions from each model
    pred1 = model1(img_tensor)
    pred2 = model2(img_tensor)
    pred3 = model3(img_tensor)

    # Apply non-maximum suppression separately for each model
    results_model.append(non_max_suppression(pred1, conf_thres=0.5, iou_thres=0.5)[0])
    results_model.append(non_max_suppression(pred2, conf_thres=0.5, iou_thres=0.5)[0])
    results_model.append(non_max_suppression(pred3, conf_thres=0.5, iou_thres=0.5)[0])


In [5]:
combined_results = []
for results in results_model:
    if results is not None:
        combined_results += results

# Sort the combined results by confidence score (if needed)
combined_results.sort(key=lambda x: x[4], reverse=True)

In [10]:
for det in combined_results:
    x1, y1, x2, y2, conf, cls = det
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    print(x1, y1, x2, y2, conf, cls)

    # Draw bounding box on the original image
    draw = ImageDraw.Draw(img)
    draw.rectangle([x1, y1, x2, y2], outline="red", width=3)

    # Print class and confidence score
    class_name = model1.names[int(cls)]  # Use the names from one of the models
    class_name2 = model2.names[int(cls)]
    class_name3 = model3.names[int(cls)]
    print(f"Class: {class_name}, Confidence: {conf:.2f}")
    print(f"Class: {class_name2}, Confidence: {conf:.2f}")
    print(f"Class: {class_name3}, Confidence: {conf:.2f}")


506 361 583 514 tensor(0.50610) tensor(74.)
Class: clock, Confidence: 0.51
Class: clock, Confidence: 0.51
Class: clock, Confidence: 0.51


In [9]:
img.show()

In [8]:
combined_results

[tensor([5.06255e+02, 3.61294e+02, 5.83037e+02, 5.14167e+02, 5.06096e-01, 7.40000e+01])]

In [13]:
results_model

[tensor([], size=(0, 6)),
 tensor([[5.06255e+02, 3.61294e+02, 5.83037e+02, 5.14167e+02, 5.06096e-01, 7.40000e+01]]),
 tensor([], size=(0, 6))]

In [15]:
img_tensor

tensor([[[[0.73333, 0.74118, 0.75686,  ..., 0.12157, 0.12157, 0.12157],
          [0.73333, 0.74118, 0.75686,  ..., 0.12157, 0.12157, 0.12157],
          [0.72549, 0.72941, 0.74118,  ..., 0.12157, 0.12157, 0.12157],
          ...,
          [0.03922, 0.03922, 0.04706,  ..., 0.12157, 0.12157, 0.12157],
          [0.05490, 0.04314, 0.01961,  ..., 0.12157, 0.12157, 0.12157],
          [0.06667, 0.05098, 0.01569,  ..., 0.12157, 0.12157, 0.12157]],

         [[0.74118, 0.74902, 0.76471,  ..., 0.12157, 0.12157, 0.12157],
          [0.74118, 0.74902, 0.76471,  ..., 0.12157, 0.12157, 0.12157],
          [0.73333, 0.73725, 0.74902,  ..., 0.12157, 0.12157, 0.12157],
          ...,
          [0.03922, 0.03922, 0.04706,  ..., 0.12157, 0.12157, 0.12157],
          [0.05098, 0.04314, 0.01961,  ..., 0.12157, 0.12157, 0.12157],
          [0.06275, 0.04706, 0.01176,  ..., 0.12157, 0.12157, 0.12157]],

         [[0.72549, 0.73333, 0.75294,  ..., 0.12157, 0.12157, 0.12157],
          [0.72549, 0.73333, 0